### Проведем инференс предобученной модели YOLO на датасете по обнаружению людей

In [1]:
import os
import math
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics import YOLO

In [2]:
os.chdir('..')

from inference_functions.yolo_inf_functions import target_boxes
from inference_functions.yolo_inf_functions import check_class
from inference_functions.yolo_inf_functions import clip_box, map50_calculate
from inference_functions.yolo_inf_functions import read_reference_video

In [3]:
TEST_IMAGES_DIR = "data/test/test"
ANNOTATIONS_CSV = "data/test/test/_annotations.csv"
CONF_THRESH = 0.001
IOU_THRESHOLD = 0.5

print("TEST_IMAGES_DIR :", TEST_IMAGES_DIR)
print("ANNOTATIONS_CSV :", ANNOTATIONS_CSV)

TEST_IMAGES_DIR : data/test/test
ANNOTATIONS_CSV : data/test/test/_annotations.csv


In [4]:
# Загружаем истинные метки областей детекции из датасета
target_boxes_dict = target_boxes(ANNOTATIONS_CSV)

Уникальных изображений в CSV: 738, общее число bbox: 2783


### Загрузка модели

In [5]:
model = YOLO("models/weights/3_yolo_users.pt")

# проверяем есть в модели класс person
person_class_idx = check_class(model)

Найден индекс класса 'person' = 0 (в model.names).


In [6]:
preds = []

def find_image_path(fname):
    '''Поиск необходимого файла'''
    p2 = TEST_IMAGES_DIR + '/' + fname
    return p2

image_list = list(target_boxes_dict.keys())

# Цикл сбора предсказаний
for fname in tqdm(image_list, desc="Running inference on test images"):

    img_path = find_image_path(fname)
    results = model.predict(source=str(img_path), conf=CONF_THRESH, imgsz=640, verbose=False)

    if len(results) == 0:
        continue
    r = results[0]

    boxes = getattr(r.boxes, "xyxy", None)
    scores = getattr(r.boxes, "conf", None)
    classes = getattr(r.boxes, "cls", None)

    if boxes is None or len(boxes) == 0:
        continue

    img_h, img_w = r.orig_shape[0], r.orig_shape[1]

    for i in range(len(boxes)):
        cls = int(classes[i].item()) if classes is not None else None
        if cls is None:
            continue
        if cls != person_class_idx:
            continue
        xyxy = boxes[i].cpu().numpy().tolist()
        score = float(scores[i].cpu().numpy().tolist()) if scores is not None else 1.0
        xyxy = clip_box(xyxy, img_w, img_h)
        preds.append({
            'image_id': fname,
            'score': score,
            'bbox': xyxy
        })

print(f"Всего предсказаний (person) собрано: {len(preds)}")

Running inference on test images: 100%|██████████| 738/738 [05:12<00:00,  2.36it/s]

Всего предсказаний (person) собрано: 17542


### Расчет MAP50 метрики

In [7]:
map50_calculate(preds, target_boxes_dict, IOU_THRESHOLD)

mAP50 для датасета = 0.4519


### Сохраняем готовое видео

In [8]:
video_path = "reference video/crowd.mp4"
save_video_path = "reference video/detected_persons/3_yolo_users_detected_crowd.mp4"

read_reference_video(model, video_path, save_video_path, person_class_idx)

Video inference: 100%|██████████| 705/705 [04:11<00:00,  2.80it/s]

Видео с боксами сохранено в reference video/detected_persons/3_yolo_users_detected_crowd.mp4
